# EXPLORATION OF DONUT HOLE WATER BOUNDARY ISSUE

## Imports, constants, and data load

In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
import os
import sys

module_path = os.path.abspath(os.path.join("../.."))
if module_path not in sys.path:
    sys.path.append(module_path)

from data_pipeline.config import settings
from data_pipeline.etl.sources.geo_utils import (
    add_tracts_for_geometries,
    get_tract_geojson,
)

In [2]:
# pull out necessary definitions from field_names.py
ORIGINAL_TRACT = "ORIGINAL_TRACT"
SCORE_N_COMMUNITIES = "Definition N (communities)"
GEOID_TRACT_FIELD = "GEOID10_TRACT"
ADJACENT_MEAN_SUFFIX = " (based on adjacency index and low income alone)"
ADJACENCY_INDEX_SUFFIX = " (average of neighbors)"
CAM_ID = '48061012305'

In [3]:
# load Census data
tract_data = get_tract_geojson()
tract_data.columns

2024-11-19 19:29:38,131 [     data_pipeline.etl.sources.geo_utils] DEBUG    Loading tract geometry data from census ETL


Index(['STATEFP10', 'COUNTYFP10', 'TRACTCE10', 'GEOID10_TRACT', 'NAME10',
       'NAMELSAD10', 'MTFCC10', 'FUNCSTAT10', 'ALAND10', 'AWATER10',
       'INTPTLAT10', 'INTPTLON10', 'geometry'],
      dtype='object')

## Look at example tract in Cameron, TX

Tract no. 48061012305 in Cameron, TX is an example of a tract that displays the donut hole water boundary issue:  all the tracts that it borders are classified as disadvantaged, and it meets the poverty threshold, yet it is being incorrectly categorized as non-disadvantaged due to its water boundary. Here we pull all adjacent tracts, compare to what we see on the map, and then look for solutions.

In [4]:
## look at census data for Cameron
cam_only_df = pd.DataFrame({GEOID_TRACT_FIELD:[CAM_ID], SCORE_N_COMMUNITIES:[False]})

cam_only_df: gpd.GeoDataFrame = tract_data.merge(
        cam_only_df, on=GEOID_TRACT_FIELD
    )
cam_only_df = cam_only_df.rename(columns={GEOID_TRACT_FIELD: ORIGINAL_TRACT})
cam_only_df

,STATEFP10,COUNTYFP10,TRACTCE10,ORIGINAL_TRACT,NAME10,NAMELSAD10,MTFCC10,FUNCSTAT10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10,geometry,Definition N (communities)
0,48,061,012305,48061012305,123.05,Census Tract 123.05,G5020,S,25920881,324996211,+26.2732070,-097.2763703,"POLYGON ((-97.22490 26.41153, -97.22436 26.411...",False


In [5]:
# get all the tracts that are adjacent to cameron
cam_adjacent_tracts: gpd.GeoDataFrame = cam_only_df.sjoin(
        tract_data, predicate="touches"
    )
cam_adjacent_tracts

,STATEFP10_left,COUNTYFP10_left,TRACTCE10_left,ORIGINAL_TRACT,NAME10_left,NAMELSAD10_left,MTFCC10_left,FUNCSTAT10_left,ALAND10_left,AWATER10_left,...,TRACTCE10_right,GEOID10_TRACT,NAME10_right,NAMELSAD10_right,MTFCC10_right,FUNCSTAT10_right,ALAND10_right,AWATER10_right,INTPTLAT10_right,INTPTLON10_right
0,48,061,012305,48061012305,123.05,Census Tract 123.05,G5020,S,25920881,324996211,...,010100,48061010100,101,Census Tract 101,G5020,S,365458278,113041841,+26.2713946,-097.4414470
0,48,061,012305,48061012305,123.05,Census Tract 123.05,G5020,S,25920881,324996211,...,950700,48489950700,9507,Census Tract 9507,G5020,S,1025415174,377000048,+26.5154317,-097.5790835
0,48,061,012305,48061012305,123.05,Census Tract 123.05,G5020,S,25920881,324996211,...,990000,48489990000,9900,Census Tract 9900,G5020,S,0,121414926,+26.5064260,-097.2240134
0,48,061,012305,48061012305,123.05,Census Tract 123.05,G5020,S,25920881,324996211,...,012700,48061012700,127,Census Tract 127,G5020,S,129622799,67725054,+25.9786218,-097.2580863
0,48,061,012305,48061012305,123.05,Census Tract 123.05,G5020,S,25920881,324996211,...,012304,48061012304,123.04,Census Tract 123.04,G5020,S,5761299,6286773,+26.0631407,-097.2184868
0,48,061,012305,48061012305,123.05,Census Tract 123.05,G5020,S,25920881,324996211,...,012301,48061012301,123.01,Census Tract 123.01,G5020,S,137622651,119430273,+26.1581170,-097.3180348
0,48,061,012305,48061012305,123.05,Census Tract 123.05,G5020,S,25920881,324996211,...,990000,48061990000,9900,Census Tract 9900,G5020,S,0,268272237,+26.1902408,-097.1473235


In [6]:
cam_adjacent_tracts.columns

Index(['STATEFP10_left', 'COUNTYFP10_left', 'TRACTCE10_left', 'ORIGINAL_TRACT',
       'NAME10_left', 'NAMELSAD10_left', 'MTFCC10_left', 'FUNCSTAT10_left',
       'ALAND10_left', 'AWATER10_left', 'INTPTLAT10_left', 'INTPTLON10_left',
       'geometry', 'Definition N (communities)', 'index_right',
       'STATEFP10_right', 'COUNTYFP10_right', 'TRACTCE10_right',
       'GEOID10_TRACT', 'NAME10_right', 'NAMELSAD10_right', 'MTFCC10_right',
       'FUNCSTAT10_right', 'ALAND10_right', 'AWATER10_right',
       'INTPTLAT10_right', 'INTPTLON10_right'],
      dtype='object')

In [7]:
cam_adjacent_tracts[['ORIGINAL_TRACT', 'GEOID10_TRACT', 
                 'AWATER10_left', 'ALAND10_left', 'AWATER10_right', 'ALAND10_right']]

,ORIGINAL_TRACT,GEOID10_TRACT,AWATER10_left,ALAND10_left,AWATER10_right,ALAND10_right
0,48061012305,48061010100,324996211,25920881,113041841,365458278
0,48061012305,48489950700,324996211,25920881,377000048,1025415174
0,48061012305,48489990000,324996211,25920881,121414926,0
0,48061012305,48061012700,324996211,25920881,67725054,129622799
0,48061012305,48061012304,324996211,25920881,6286773,5761299
0,48061012305,48061012301,324996211,25920881,119430273,137622651
0,48061012305,48061990000,324996211,25920881,268272237,0


In [8]:
# # compare to what I see on the map for Cameron:

# 48489950700 Willacy County
# 48061010100 Cameron County (contains Laguna Atascosa)
# 48061012301 Cameron County (contains KPIL)
# 48061012304 Cameron County (contains Port Isabel)
# 48061012700 Cameron County (contains Boca Chica Village)

# # what about 48489990000 and 48061990000?
# both have ALAND10 = 0
# and both tract IDs end in 9990000

Proposed solution: remove tracts where ALAND10=10 (ie water-only tracts)

## Compare water-only tracts to tracts with land area

In [9]:
# How many of these water-only tracts exist?
len(tract_data[tract_data.ALAND10==0])

367

In [10]:
# make a copy of the tract data
tract_explore = tract_data.copy()

# add bool for whether tract has land area
tract_explore['water_only'] = tract_explore.ALAND10==0

# preview water tracts
tract_explore[tract_explore.water_only]

,STATEFP10,COUNTYFP10,TRACTCE10,GEOID10_TRACT,NAME10,NAMELSAD10,MTFCC10,FUNCSTAT10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10,geometry,water_only
808,36,063,990000,36063990000,9900,Census Tract 9900,G5020,S,0,1550634668,+43.4569085,-078.7926530,"POLYGON ((-78.46554 43.37337, -78.47053 43.372...",True
1015,36,085,008900,36085008900,89,Census Tract 89,G5020,S,0,100710,+40.6469814,-074.1024106,"POLYGON ((-74.10504 40.64808, -74.10025 40.648...",True
1149,36,085,990100,36085990100,9901,Census Tract 9901,G5020,S,0,80255151,+40.5255512,-074.1085829,"POLYGON ((-74.25482 40.49390, -74.25308 40.491...",True
1219,36,013,990000,36013990000,9900,Census Tract 9900,G5020,S,0,1015062614,+42.5006582,-079.5141161,"POLYGON ((-79.24982 42.53745, -79.24982 42.537...",True
1226,36,075,990000,36075990000,9900,Census Tract 9900,G5020,S,0,724224875,+43.5955454,-076.4442549,"POLYGON ((-76.55001 43.45923, -76.55064 43.458...",True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73379,23,015,990000,23015990000,9900,Census Tract 9900,G5020,S,0,375299033,+43.8029690,-069.4652520,"POLYGON ((-69.68695 43.81613, -69.68695 43.815...",True
73399,23,031,990100,23031990100,9901,Census Tract 9901,G5020,S,0,567248819,+43.1483531,-070.4998364,"POLYGON ((-70.22209 43.46688, -70.22432 43.464...",True
73509,23,023,990000,23023990000,9900,Census Tract 9900,G5020,S,0,161372101,+43.7084463,-069.7721513,"POLYGON ((-69.69756 43.81594, -69.69755 43.813...",True
73527,23,013,990000,23013990000,9900,Census Tract 9900,G5020,S,0,1636919697,+43.9538894,-068.9327255,"POLYGON ((-68.73563 44.11917, -68.73179 44.117...",True


In [11]:
# confirm we can get the tract code stored in TRACTCE10 by taking last 6 chars of GEOID10_TRACT
# (because input df may not contain the isolated tract code column)
tract_explore['tract_code'] = tract_explore.GEOID10_TRACT.apply(lambda x: x[-6:])
assert all(tract_explore.tract_code==tract_explore.TRACTCE10)

In [12]:
# convert tract codes to ints to check ranges
tract_explore['tract_code_num'] = tract_explore['tract_code'].apply(lambda x: int(x))
tract_explore[['GEOID10_TRACT', 'TRACTCE10', 'tract_code', 'tract_code_num']].head()

,GEOID10_TRACT,TRACTCE10,tract_code,tract_code_num
0,20071958100,958100,958100,958100
1,20175965600,965600,965600,965600
2,20175965700,965700,965700,965700
3,20043020300,020300,020300,20300
4,20043020200,020200,020200,20200


Per Census documentation:

- 000100 to 989900—Basic number range for census tracts
- 990000 to 990099—Basic number for census tracts in water areas
- 990100 to 998900—Basic number range for census tracts

source: https://www2.census.gov/geo/pdfs/maps-data/data/tiger/tgrshp2017/TGRSHP2017_TechDoc_Ch3.pdf  (page 3-26)

In [13]:
def in_water_range(x):
    if x >= 990000 and x <= 990099:
        return True
    return False

tract_explore['tract_in_water_range'] = tract_explore['tract_code_num'].apply(in_water_range)
tract_explore[['GEOID10_TRACT', 'TRACTCE10', 'tract_code', 'tract_code_num', 'tract_in_water_range']].head()

,GEOID10_TRACT,TRACTCE10,tract_code,tract_code_num,tract_in_water_range
0,20071958100,958100,958100,958100,False
1,20175965600,965600,965600,965600,False
2,20175965700,965700,965700,965700,False
3,20043020300,020300,020300,20300,False
4,20043020200,020200,020200,20200,False


In [14]:
# look at water-only tracts
tract_explore[tract_explore.water_only]\
    [['GEOID10_TRACT', 'TRACTCE10', 'tract_code', 'tract_code_num', 'tract_in_water_range']]

,GEOID10_TRACT,TRACTCE10,tract_code,tract_code_num,tract_in_water_range
808,36063990000,990000,990000,990000,True
1015,36085008900,008900,008900,8900,False
1149,36085990100,990100,990100,990100,False
1219,36013990000,990000,990000,990000,True
1226,36075990000,990000,990000,990000,True
...,...,...,...,...,...
73379,23015990000,990000,990000,990000,True
73399,23031990100,990100,990100,990100,False
73509,23023990000,990000,990000,990000,True
73527,23013990000,990000,990000,990000,True


Some of the tracts with ALAND10=0 have tract numbers not in the water-only range.

In [15]:
tract_explore.groupby('water_only').agg({'GEOID10_TRACT': 'count', 
                                         'tract_in_water_range': ['sum', 'mean']})

GEOID10_TRACT tract_in_water_range          
                   count                  sum      mean
water_only                                             
False              73767                    0  0.000000
True                 367                  227  0.618529

There are no tracts with land area that have tract codes in the water-only range. But about 38% of tracts with 0 land area have normal tract codes outside the water range.

In [16]:
# view the exceptions (no land area, but tract code not in water range)
water_exceptions = tract_explore[(tract_explore.water_only==True) & (tract_explore.tract_in_water_range==False)]
water_exceptions

,STATEFP10,COUNTYFP10,TRACTCE10,GEOID10_TRACT,NAME10,NAMELSAD10,MTFCC10,FUNCSTAT10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10,geometry,water_only,tract_code,tract_code_num,tract_in_water_range
1015,36,085,008900,36085008900,89,Census Tract 89,G5020,S,0,100710,+40.6469814,-074.1024106,"POLYGON ((-74.10504 40.64808, -74.10025 40.648...",True,008900,8900,False
1149,36,085,990100,36085990100,9901,Census Tract 9901,G5020,S,0,80255151,+40.5255512,-074.1085829,"POLYGON ((-74.25482 40.49390, -74.25308 40.491...",True,990100,990100,False
2840,36,047,990100,36047990100,9901,Census Tract 9901,G5020,S,0,17793514,+40.5649933,-074.0148865,"POLYGON ((-74.00620 40.59437, -74.00619 40.594...",True,990100,990100,False
3095,36,059,990100,36059990100,9901,Census Tract 9901,G5020,S,0,13672168,+40.8790568,-073.7020503,"POLYGON ((-73.76050 40.84184, -73.76102 40.841...",True,990100,990100,False
3096,36,059,990200,36059990200,9902,Census Tract 9902,G5020,S,0,27086403,+40.9078457,-073.6526612,"POLYGON ((-73.65151 40.88524, -73.66481 40.875...",True,990200,990200,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68946,15,009,990200,15009990200,9902,Census Tract 9902,G5020,S,0,1643042883,+20.6128423,-156.5486800,"POLYGON ((-156.70541 20.82632, -156.70658 20.8...",True,990200,990200,False
68949,15,009,991200,15009991200,9912,Census Tract 9912,G5020,S,0,482573725,+20.9476085,-156.9795251,"POLYGON ((-157.11121 20.87500, -157.11725 20.8...",True,991200,991200,False
70339,39,093,990200,39093990200,9902,Census Tract 9902,G5020,S,0,1094173093,+41.6347746,-082.1526215,"POLYGON ((-82.16712 41.47534, -82.16943 41.474...",True,990200,990200,False
70979,39,043,990100,39043990100,9901,Census Tract 9901,G5020,S,0,853315204,+41.5559059,-082.5241364,"POLYGON ((-82.51199 41.39475, -82.51472 41.395...",True,990100,990100,False


In [17]:
# print list of "normal" water-only IDs to put into compare tool
water_normals = tract_explore[(tract_explore.water_only==True) & (tract_explore.tract_in_water_range==True)]
print(', '.join(water_normals.GEOID10_TRACT.values))

36063990000, 36013990000, 36075990000, 36055990000, 36045990001, 36029990000, 36073990000, 41007990000, 41039990000, 41019990000, 01003990000, 01097990000, 17031990000, 17097990000, 10001990000, 10005990000, 06083990000, 06053990000, 06061990000, 06015990000, 06079990000, 06013990000, 06017990000, 06001990000, 51710990000, 26157990000, 26011990000, 26001990000, 26141990000, 26019990000, 26069990000, 26053990000, 26147990000, 26153990000, 26063990000, 26159990000, 26033990000, 26005990000, 26103990000, 26061990000, 26003990000, 26095990000, 26041990000, 26031990000, 26009990000, 26139990000, 26047990000, 26109990000, 26007990000, 26021990000, 26089990000, 26017990000, 26115990000, 26151990000, 26055990000, 26127990000, 26097990000, 26121990000, 26105990000, 26101990000, 26029990000, 66010990000, 27031990000, 69100990000, 69110990000, 69120990000, 28045990000, 28059990000, 28047990000, 53031990000, 53061990002, 53027990000, 12009990000, 12017990000, 12037990000, 12011990000, 12087990000,

In [18]:
# print list of ALAND10=0 IDs to put into compare tool
print(', '.join(tract_explore[tract_explore.water_only==True].GEOID10_TRACT.values))

36063990000, 36085008900, 36085990100, 36013990000, 36075990000, 36055990000, 36045990001, 36047990100, 36059990100, 36059990200, 36059990301, 36059990302, 36059990400, 36081990100, 36029990000, 36073990000, 36103990100, 36117990100, 36011990200, 41007990000, 41057990100, 41039990000, 41041990100, 41015990101, 41011990101, 41019990000, 01003990000, 01097990000, 17031990000, 17097990000, 37031990100, 37031990200, 37053990100, 37019990100, 37055990100, 37055990200, 37129990100, 37095990100, 37095990200, 37141990100, 37137990100, 37133990100, 10001990000, 10005990000, 10003990100, 06083990000, 06053990000, 06061990000, 06059990100, 06097990100, 06045990100, 06073990100, 06041990100, 06015990000, 06081990100, 06079990000, 06013990000, 06111990100, 06017990000, 06001990000, 06075990100, 06087990100, 06023990100, 06037990200, 06037990300, 06037990100, 51103990100, 51115990100, 51650990100, 51131990100, 51735990100, 51810990100, 51133990100, 51001990100, 51001990200, 51119990100, 51199990100,

Tract comparison tool shoes that 21 of the 140 "water exception" tracts are in Hawaii. California next highest with 13. Puerto Rico has 12. (No other territories have "water exception" tracts.)

## Simulate calculate_tract_adjacency_scores()

In [19]:
def calculate_tract_adjacency_scores(
    df: pd.DataFrame, score_column: str, tract_data
) -> pd.DataFrame:
    """Calculate the mean score of each tract in df based on its neighbors

    Args:
        df (pandas.DataFrame): A dataframe with at least the following columns:
          * field_names.GEOID_TRACT_FIELD
          * score_column

        score_column (str): The name of the column that contains the scores
                            to average
                            
        tract_data (GeoDataFrame): tract data normally loaded in first line of 
                                    function: tract_data = get_tract_geojson()
    Returns :
        tuple containing final returned df in actual function, as well as intermediates:
            - returned_donut_bools (pandas.DataFrame): A dataframe with two columns:
              * field_names.GEOID_TRACT_FIELD
              * {score_column}_ADJACENT_MEAN, which is the average of score_column for
                each tract that touches the tract identified
                in field_names.GEOID_TRACT_FIELD
              NB: this is the df that gets returned in the actual function
            - df (pandas.DataFrame): input df after merging with Census data
            - adjacent_tracts (pandas.DataFrame): adjacency df
    """

    df: gpd.GeoDataFrame = tract_data.merge(
        df, on=GEOID_TRACT_FIELD
    )
    df = df.rename(columns={GEOID_TRACT_FIELD: ORIGINAL_TRACT})


    adjacent_tracts: gpd.GeoDataFrame = df.sjoin(
        tract_data, predicate="touches"
    )


    returned_donut_bools = (
        adjacent_tracts.groupby(GEOID_TRACT_FIELD)[[score_column]]
        .mean()
        .reset_index()
        .rename(
            columns={
                score_column: f"{score_column}{ADJACENCY_INDEX_SUFFIX}",
            }
        )
    )
    
    return (returned_donut_bools, df, adjacent_tracts)


In [20]:
# save tract ids that are adjacent to cameron so we can simulate 
# passing them into calculate_tract_adjacency_scores() via df arg
adjacent_track_id_list_cam = list(cam_adjacent_tracts.GEOID10_TRACT.unique())
adjacent_track_id_list_cam

['48061010100',
 '48489950700',
 '48489990000',
 '48061012700',
 '48061012304',
 '48061012301',
 '48061990000']

In [21]:
# df is passed into the function, must contain the score column we're looking at
# (in this case, SCORE_N_COMMUNITIES)
# here we pass in our water boundary issue example in Cameron TX pluss all its adjacencies.
# initialize disadvantaged bool as FALSE for our example, and TRUE for others.
df_cam_plus = pd.DataFrame({GEOID_TRACT_FIELD:[CAM_ID]+adjacent_track_id_list_cam, 
                   SCORE_N_COMMUNITIES:[False]+([True]*len(adjacent_track_id_list_cam))})

# set disadvantaged bool to False for water tracts
df_cam_plus.loc[df_cam_plus[GEOID_TRACT_FIELD].isin(['48489990000', '48061990000']), SCORE_N_COMMUNITIES]=False

df_cam_plus

,GEOID10_TRACT,Definition N (communities)
0,48061012305,False
1,48061010100,True
2,48489950700,True
3,48489990000,False
4,48061012700,True
5,48061012304,True
6,48061012301,True
7,48061990000,False


### Simulate v1 run of of calculate_tract_adjacency_scores()

In [22]:
returned_donut_bools__v1, df_census__v1, adjacent_tracts__v1 = calculate_tract_adjacency_scores(
    df=df_cam_plus, score_column=SCORE_N_COMMUNITIES, tract_data=tract_data
)

In [23]:
adjacent_tracts__v1

,STATEFP10_left,COUNTYFP10_left,TRACTCE10_left,ORIGINAL_TRACT,NAME10_left,NAMELSAD10_left,MTFCC10_left,FUNCSTAT10_left,ALAND10_left,AWATER10_left,...,TRACTCE10_right,GEOID10_TRACT,NAME10_right,NAMELSAD10_right,MTFCC10_right,FUNCSTAT10_right,ALAND10_right,AWATER10_right,INTPTLAT10_right,INTPTLON10_right
0,48,489,990000,48489990000,9900,Census Tract 9900,G5020,S,0,121414926,...,012305,48061012305,123.05,Census Tract 123.05,G5020,S,25920881,324996211,+26.2732070,-097.2763703
1,48,489,950700,48489950700,9507,Census Tract 9507,G5020,S,1025415174,377000048,...,012305,48061012305,123.05,Census Tract 123.05,G5020,S,25920881,324996211,+26.2732070,-097.2763703
2,48,061,990000,48061990000,9900,Census Tract 9900,G5020,S,0,268272237,...,012305,48061012305,123.05,Census Tract 123.05,G5020,S,25920881,324996211,+26.2732070,-097.2763703
4,48,061,012304,48061012304,123.04,Census Tract 123.04,G5020,S,5761299,6286773,...,012305,48061012305,123.05,Census Tract 123.05,G5020,S,25920881,324996211,+26.2732070,-097.2763703
5,48,061,010100,48061010100,101,Census Tract 101,G5020,S,365458278,113041841,...,012305,48061012305,123.05,Census Tract 123.05,G5020,S,25920881,324996211,+26.2732070,-097.2763703
6,48,061,012700,48061012700,127,Census Tract 127,G5020,S,129622799,67725054,...,012305,48061012305,123.05,Census Tract 123.05,G5020,S,25920881,324996211,+26.2732070,-097.2763703
7,48,061,012301,48061012301,123.01,Census Tract 123.01,G5020,S,137622651,119430273,...,012305,48061012305,123.05,Census Tract 123.05,G5020,S,25920881,324996211,+26.2732070,-097.2763703
0,48,489,990000,48489990000,9900,Census Tract 9900,G5020,S,0,121414926,...,950700,48489950700,9507,Census Tract 9507,G5020,S,1025415174,377000048,+26.5154317,-097.5790835
3,48,061,012305,48061012305,123.05,Census Tract 123.05,G5020,S,25920881,324996211,...,950700,48489950700,9507,Census Tract 9507,G5020,S,1025415174,377000048,+26.5154317,-097.5790835
5,48,061,010100,48061010100,101,Census Tract 101,G5020,S,365458278,113041841,...,950700,48489950700,9507,Census Tract 9507,G5020,S,1025415174,377000048,+26.5154317,-097.5790835


In [24]:
### check for duplicates
assert len(adjacent_tracts__v1[['ORIGINAL_TRACT', 'GEOID10_TRACT']].drop_duplicates()) == len(adjacent_tracts__v1[['ORIGINAL_TRACT', 'GEOID10_TRACT']])

In [25]:
returned_donut_bools__v1

,GEOID10_TRACT,Definition N (communities) (average of neighbors)
0,48061010100,0.666667
1,48061010201,1.000000
2,48061010203,1.000000
3,48061010800,1.000000
4,48061011400,1.000000
5,48061012200,1.000000
6,48061012301,0.666667
7,48061012304,0.666667
8,48061012305,0.714286
9,48061012607,1.000000


In [26]:
# check result for our example tract
returned_donut_bools__v1[returned_donut_bools__v1.GEOID10_TRACT==CAM_ID]

,GEOID10_TRACT,Definition N (communities) (average of neighbors)
8,48061012305,0.714286


Demonstrates that because 2 of the 7 adjacent tracts have meaningless water-only non-disadvantaged status, the average of neighbors is less than 1. This is why our example tract isn't being classified as disadvantaged in v1.

### Simulate proposed v2 implementations of calculate_tract_adjacency_scores()

Testing three ways to do this: filtering tract_data, filtering df, and filtering both. For the first two ways, we'll test both land area and tract ID range methods.

In [27]:
def view_results(returned_donut_bools, post_merge_df, adjacent_tracts):
    # function to print what we need to see after each experiment
    
    print(f'Length of input df after Census merge: {len(post_merge_df)}') 
    print(f'That is {len(df_census__v1) - len(post_merge_df)} less than in v1')
    
    print(f'\nLength of adjacency frame: {len(adjacent_tracts)}')
    print(f'That is {len(adjacent_tracts__v1) - len(adjacent_tracts)} less than in v1')
    
    n_dupes = len(adjacent_tracts[['ORIGINAL_TRACT', 'GEOID10_TRACT']])\
                - len(adjacent_tracts[['ORIGINAL_TRACT', 'GEOID10_TRACT']].drop_duplicates())
    if n_dupes>0:
        print("ALERT: duplicates present in adjacency frame!")

    
    print(f'\nLength of returned frame with donut bools: {len(returned_donut_bools)}')
    if len(returned_donut_bools__v1) == len(returned_donut_bools):
        print('Returning same number of final bools as in v1')
    else:
        print(f'ALERT: returning {len(returned_donut_bools__v1) - len(returned_donut_bools)} less than in v1')

    cam_return = returned_donut_bools[returned_donut_bools.GEOID10_TRACT==CAM_ID]
    display(cam_return)

    if cam_return['Definition N (communities) (average of neighbors)'].values[0]==1:
        print('SUCCESSFULLY RE-ASSIGNING CAMERON, TX EXAMPLE TO DISADVANTAGED STATUS')
    else:
        print('ALERT: CAMERON, TX EXAMPLE IS STILL BEING MARKED NON-DISADVANTAGED')

#### Implementation 1a: Filter water-only tracts out of tract_data using ALAND10

In [28]:
tract_data_land_area_only = tract_data.copy()
tract_data_land_area_only = tract_data_land_area_only[tract_data_land_area_only.ALAND10>0]

returned_donut_bools__filter_tracts_aland10,\
df_census__filter_tracts_aland10,\
adjacent_tracts__filter_tracts_aland10 =calculate_tract_adjacency_scores(
        df=df_cam_plus, 
        score_column=SCORE_N_COMMUNITIES, 
        tract_data=tract_data_land_area_only
)

view_results(returned_donut_bools=returned_donut_bools__filter_tracts_aland10, 
             post_merge_df=df_census__filter_tracts_aland10, 
             adjacent_tracts=adjacent_tracts__filter_tracts_aland10)

Length of input df after Census merge: 6
That is 2 less than in v1

Length of adjacency frame: 40
That is 13 less than in v1

Length of returned frame with donut bools: 23
ALERT: returning 3 less than in v1


,GEOID10_TRACT,Definition N (communities) (average of neighbors)
8,48061012305,1.0


SUCCESSFULLY RE-ASSIGNING CAMERON, TX EXAMPLE TO DISADVANTAGED STATUS


#### Implementation 1b: Filter water-only tracts out of df using tract range

In [29]:
def full_geo_id_to_water_range_bool(x:str):
    num_x = int(x[-6:])
    return(in_water_range(num_x))

In [30]:
tract_data_non_water_range_only = tract_data.copy()
tract_data_non_water_range_only = tract_data_non_water_range_only[tract_data_non_water_range_only.GEOID10_TRACT\
                                            .apply(full_geo_id_to_water_range_bool)\
                                                ==False]

returned_donut_bools__filter_tracts_id_range,\
df_census__filter_tracts_id_range,\
adjacent_tracts__filter_tracts_id_range =calculate_tract_adjacency_scores(
        df=df_cam_plus, 
        score_column=SCORE_N_COMMUNITIES, 
        tract_data=tract_data_non_water_range_only
)

view_results(returned_donut_bools=returned_donut_bools__filter_tracts_id_range, 
             post_merge_df=df_census__filter_tracts_id_range, 
             adjacent_tracts=adjacent_tracts__filter_tracts_id_range)

Length of input df after Census merge: 6
That is 2 less than in v1

Length of adjacency frame: 40
That is 13 less than in v1

Length of returned frame with donut bools: 23
ALERT: returning 3 less than in v1


,GEOID10_TRACT,Definition N (communities) (average of neighbors)
8,48061012305,1.0


SUCCESSFULLY RE-ASSIGNING CAMERON, TX EXAMPLE TO DISADVANTAGED STATUS


#### Implementation 2a: Filter water-only tracts out of df using ALAND10

Note: we can't filter the input df based on ALAND10 using the standard calculate_tract_adjacency_scores() function, because it doesn't have the ALAND10 column until after it is merged with tract data. Need to write a new test function for this method.

In [31]:
def calculate_tract_adjacency_scores__filter_df_aland10(
    df: pd.DataFrame, score_column: str, tract_data
) -> pd.DataFrame:
    """Calculate the mean score of each tract in df based on its neighbors

    Args:
        df (pandas.DataFrame): A dataframe with at least the following columns:
          * field_names.GEOID_TRACT_FIELD
          * score_column

        score_column (str): The name of the column that contains the scores
                            to average
                            
        tract_data (GeoDataFrame): tract data normally loaded in first line of 
                                    function: tract_data = get_tract_geojson()
    Returns :
        tuple containing final returned df in actual function, as well as intermediates:
            - returned_donut_bools (pandas.DataFrame): A dataframe with two columns:
              * field_names.GEOID_TRACT_FIELD
              * {score_column}_ADJACENT_MEAN, which is the average of score_column for
                each tract that touches the tract identified
                in field_names.GEOID_TRACT_FIELD
              NB: this is the df that gets returned in the actual function
            - df (pandas.DataFrame): input df after merging with Census data
            - adjacent_tracts (pandas.DataFrame): adjacency df
    """

    df: gpd.GeoDataFrame = tract_data.merge(
        df, on=GEOID_TRACT_FIELD
    )
    df = df.rename(columns={GEOID_TRACT_FIELD: ORIGINAL_TRACT})
    
    
    # remove water areas from input frame
    df = df[df['ALAND10']>0]


    adjacent_tracts: gpd.GeoDataFrame = df.sjoin(
        tract_data, predicate="touches"
    )


    returned_donut_bools = (
        adjacent_tracts.groupby(GEOID_TRACT_FIELD)[[score_column]]
        .mean()
        .reset_index()
        .rename(
            columns={
                score_column: f"{score_column}{ADJACENCY_INDEX_SUFFIX}",
            }
        )
    )
    
    return (returned_donut_bools, df, adjacent_tracts)


In [32]:
returned_donut_bools__filter_df_aland10,\
df_census__filter_df_aland10,\
adjacent_tracts__filter_df_aland10 = calculate_tract_adjacency_scores__filter_df_aland10(
        df=df_cam_plus, 
        score_column=SCORE_N_COMMUNITIES, 
        tract_data=tract_data
)

view_results(returned_donut_bools=returned_donut_bools__filter_df_aland10, 
             post_merge_df=df_census__filter_df_aland10, 
             adjacent_tracts=adjacent_tracts__filter_df_aland10)

Length of input df after Census merge: 6
That is 2 less than in v1

Length of adjacency frame: 45
That is 8 less than in v1

Length of returned frame with donut bools: 26
Returning same number of final bools as in v1


,GEOID10_TRACT,Definition N (communities) (average of neighbors)
8,48061012305,1.0


SUCCESSFULLY RE-ASSIGNING CAMERON, TX EXAMPLE TO DISADVANTAGED STATUS


#### Implementation 2b: Filter water-only tracts out of df using tract range

In [33]:
df_cam_plus_no_water = df_cam_plus.copy()
df_cam_plus_no_water = df_cam_plus_no_water[df_cam_plus_no_water.GEOID10_TRACT\
                                            .apply(full_geo_id_to_water_range_bool)\
                                                ==False]

returned_donut_bools__filter_df_id_range,\
df_census__filter_df_id_range,\
adjacent_tracts__filter_df_id_range = calculate_tract_adjacency_scores(
        df=df_cam_plus_no_water, 
        score_column=SCORE_N_COMMUNITIES, 
        tract_data=tract_data
)

view_results(returned_donut_bools=returned_donut_bools__filter_df_id_range, 
             post_merge_df=df_census__filter_df_id_range, 
             adjacent_tracts=adjacent_tracts__filter_df_id_range)

Length of input df after Census merge: 6
That is 2 less than in v1

Length of adjacency frame: 45
That is 8 less than in v1

Length of returned frame with donut bools: 26
Returning same number of final bools as in v1


,GEOID10_TRACT,Definition N (communities) (average of neighbors)
8,48061012305,1.0


SUCCESSFULLY RE-ASSIGNING CAMERON, TX EXAMPLE TO DISADVANTAGED STATUS


#### Implementation 3: Filter water-only tracts out of both tract_data and df

Note: test id method for simplicity

In [34]:
returned_donut_bools__filter_both,\
df_census__filter_both,\
adjacent_tracts__filter_both =calculate_tract_adjacency_scores(
        df=df_cam_plus_no_water, 
        score_column=SCORE_N_COMMUNITIES, 
        tract_data=tract_data_non_water_range_only
)

view_results(returned_donut_bools=returned_donut_bools__filter_both, 
             post_merge_df=df_census__filter_both, 
             adjacent_tracts=adjacent_tracts__filter_both)

Length of input df after Census merge: 6
That is 2 less than in v1

Length of adjacency frame: 40
That is 13 less than in v1

Length of returned frame with donut bools: 23
ALERT: returning 3 less than in v1


,GEOID10_TRACT,Definition N (communities) (average of neighbors)
8,48061012305,1.0


SUCCESSFULLY RE-ASSIGNING CAMERON, TX EXAMPLE TO DISADVANTAGED STATUS


#### Summary of results: All solutions successfully re-assigned our example tract to disadvantaged status. Only implementations 2a & 2b (filtering on the dataframe) also preserved the number of tracts that the function returns

## Compare full scoring runs from different methods

In [35]:
# read in CSV with deltas from implementation 1a 
# (water areas filtered from tract_data via ALAND10)
deltas__tract_area = pd.read_csv('../data/tmp/Comparator/Score/deltas__tract_area.csv')

# rename key columns
deltas__tract_area.rename(columns = {'Unnamed: 0': 'GEOID10_TRACT',
                                'Definition N community, including adjacency index tracts': 'final_dac_prod',
                                'Definition N community, including adjacency index tracts.1': 'final_dac_local',
                                'Definition N (communities) (average of neighbors)': 'dac_adj_avg_prod',
                                'Definition N (communities) (average of neighbors).1': 'dac_adj_avg_local',
                                'Is the tract surrounded by disadvantaged communities?': 'is_donut_hole_prod',
                                'Is the tract surrounded by disadvantaged communities?.1': 'is_donut_hole_local'
                                },
                    inplace=True) 

# drop first two rows, which old column information
deltas__tract_area.drop(index=[0,1], inplace=True)

# create bool to store whether final DAC designation was updated
deltas__tract_area['class_change'] = deltas__tract_area.final_dac_prod!=deltas__tract_area.final_dac_local

# set class change bool to false where both designations are null
deltas__tract_area.loc[((deltas__tract_area.final_dac_prod.isna()==True)\
                       & (deltas__tract_area.final_dac_local.isna()==True)), 
                    'class_change'] = False

# view tracts that had their status updated with this method
deltas__tract_area[deltas__tract_area.class_change][['GEOID10_TRACT', 
                                                     'dac_adj_avg_prod', 'dac_adj_avg_local',
                                                     'is_donut_hole_prod', 'is_donut_hole_local',
                                                     'final_dac_prod', 'final_dac_local',
                                                     'class_change']]

,GEOID10_TRACT,dac_adj_avg_prod,dac_adj_avg_local,is_donut_hole_prod,is_donut_hole_local,final_dac_prod,final_dac_local,class_change
332,12087971001,0.67,1.0,False,True,False,True,True
370,12101030202,0.83,1.0,False,True,False,True,True
376,12101030303,0.83,1.0,False,True,False,True,True
419,12127081102,0.75,1.0,False,True,False,True,True
806,26069000200,0.86,1.0,False,True,False,True,True
812,26083000100,0.33,1.0,False,True,False,True,True
912,28047001400,0.88,1.0,False,True,False,True,True
1138,39007000602,0.75,1.0,False,True,False,True,True
1654,41041950303,0.75,1.0,False,True,False,True,True
1739,48061012305,0.71,1.0,False,True,False,True,True


In [36]:
# read in CSV with deltas from implementation 2a 
# (water areas filtered from input df via ALAND10)
deltas__df_area = pd.read_csv('../data/tmp/Comparator/Score/deltas__df_area.csv')

# rename key columns
deltas__df_area.rename(columns = {'Unnamed: 0': 'GEOID10_TRACT',
                                'Definition N community, including adjacency index tracts': 'final_dac_prod',
                                'Definition N community, including adjacency index tracts.1': 'final_dac_local',
                                'Definition N (communities) (average of neighbors)': 'dac_adj_avg_prod',
                                'Definition N (communities) (average of neighbors).1': 'dac_adj_avg_local',
                                'Is the tract surrounded by disadvantaged communities?': 'is_donut_hole_prod',
                                'Is the tract surrounded by disadvantaged communities?.1': 'is_donut_hole_local'
                                },
                    inplace=True) 

# drop first two rows, which old column information
deltas__df_area.drop(index=[0,1], inplace=True)

# create bool to store whether final DAC designation was updated
deltas__df_area['class_change'] = deltas__df_area.final_dac_prod!=deltas__df_area.final_dac_local

# set class change bool to false where both designations are null
deltas__df_area.loc[((deltas__df_area.final_dac_prod.isna()==True)\
                       & (deltas__df_area.final_dac_local.isna()==True)), 
                    'class_change'] = False

# view tracts that had their status updated with this method
deltas__df_area[deltas__df_area.class_change][['GEOID10_TRACT', 
                                           'dac_adj_avg_prod', 'dac_adj_avg_local',
                                           'is_donut_hole_prod', 'is_donut_hole_local',
                                            'final_dac_prod', 'final_dac_local',
                                            'class_change']]

,GEOID10_TRACT,dac_adj_avg_prod,dac_adj_avg_local,is_donut_hole_prod,is_donut_hole_local,final_dac_prod,final_dac_local,class_change
312,12087971001,0.67,1.0,False,True,False,True,True
346,12101030202,0.83,1.0,False,True,False,True,True
352,12101030303,0.83,1.0,False,True,False,True,True
388,12127081102,0.75,1.0,False,True,False,True,True
741,26069000200,0.86,1.0,False,True,False,True,True
747,26083000100,0.33,1.0,False,True,False,True,True
836,28047001400,0.88,1.0,False,True,False,True,True
1044,39007000602,0.75,1.0,False,True,False,True,True
1559,41041950303,0.75,1.0,False,True,False,True,True
1643,48061012305,0.71,1.0,False,True,False,True,True


In [37]:
# read in CSV with deltas from implementation 2b 
# (water areas filtered from input df via IDs)
deltas__df_id = pd.read_csv('../data/tmp/Comparator/Score/deltas__df_id.csv')

# rename key columns
deltas__df_id.rename(columns = {'Unnamed: 0': 'GEOID10_TRACT',
                                'Definition N community, including adjacency index tracts': 'final_dac_prod',
                                'Definition N community, including adjacency index tracts.1': 'final_dac_local',
                                'Definition N (communities) (average of neighbors)': 'dac_adj_avg_prod',
                                'Definition N (communities) (average of neighbors).1': 'dac_adj_avg_local',
                                'Is the tract surrounded by disadvantaged communities?': 'is_donut_hole_prod',
                                'Is the tract surrounded by disadvantaged communities?.1': 'is_donut_hole_local'
                                },
                    inplace=True) 

# drop first two rows, which old column information
deltas__df_id.drop(index=[0,1], inplace=True)

# create bool to store whether final DAC designation was updated
deltas__df_id['class_change'] = deltas__df_id.final_dac_prod!=deltas__df_id.final_dac_local

# set class change bool to false where both designations are null
deltas__df_id.loc[((deltas__df_id.final_dac_prod.isna()==True)\
                       & (deltas__df_id.final_dac_local.isna()==True)), 
                    'class_change'] = False

# view tracts that had their status updated with this method
deltas__df_id[deltas__df_id.class_change][['GEOID10_TRACT', 
                                           'dac_adj_avg_prod', 'dac_adj_avg_local',
                                           'is_donut_hole_prod', 'is_donut_hole_local',
                                            'final_dac_prod', 'final_dac_local',
                                            'class_change']]

,GEOID10_TRACT,dac_adj_avg_prod,dac_adj_avg_local,is_donut_hole_prod,is_donut_hole_local,final_dac_prod,final_dac_local,class_change
228,12087971001,0.67,1.0,False,True,False,True,True
261,12101030202,0.83,1.0,False,True,False,True,True
267,12101030303,0.83,1.0,False,True,False,True,True
292,12127081102,0.75,1.0,False,True,False,True,True
570,26069000200,0.86,1.0,False,True,False,True,True
648,28047001400,0.88,1.0,False,True,False,True,True
755,39007000602,0.75,1.0,False,True,False,True,True
1281,48061012305,0.71,1.0,False,True,False,True,True
1388,55101000100,0.75,1.0,False,True,False,True,True
1470,72037160100,0.83,1.0,False,True,False,True,True


In [38]:
# print list of tract IDs where status was updated by method 1a
updated_ids__tract_area = deltas__tract_area[deltas__tract_area.class_change]['GEOID10_TRACT'].values
print(', '.join(updated_ids__tract_area))

12087971001, 12101030202, 12101030303, 12127081102, 26069000200, 26083000100, 28047001400, 39007000602, 41041950303, 48061012305, 51001090600, 51131930200, 55101000100, 72013301000, 72037160100


In [39]:
# print list of tract IDs where status was updated by method 2a
updated_ids__df_area = deltas__df_area[deltas__df_area.class_change]['GEOID10_TRACT'].values
print(', '.join(updated_ids__df_area))

12087971001, 12101030202, 12101030303, 12127081102, 26069000200, 26083000100, 28047001400, 39007000602, 41041950303, 48061012305, 51001090600, 51131930200, 55101000100, 72013301000, 72037160100


In [40]:
# print list of tract IDs where status was updated by method 2b
updated_ids__df_id = deltas__df_id[deltas__df_id.class_change]['GEOID10_TRACT'].values
print(', '.join(updated_ids__df_id))

12087971001, 12101030202, 12101030303, 12127081102, 26069000200, 28047001400, 39007000602, 48061012305, 55101000100, 72037160100


In [41]:
# how many tracts updated by method 1a?
len(updated_ids__tract_area)

15

In [42]:
# how many tracts updated by method 2a?
len(updated_ids__df_area)

15

In [43]:
# how many tracts updated by method 2b?
len(updated_ids__df_id)

10

In [44]:
# how many tracts are changed by both 1a and 2a?
len(set(updated_ids__tract_area) & set(updated_ids__df_area))

15

In [45]:
# how many tracts are changed by all 3?
len(set(updated_ids__tract_area) & set(updated_ids__df_id))

10

Implementations 1a & 2a result in the same designations. They makes all the same changes as implementation 2b, plus 5 more.

In [46]:
# view the tracts that are changed with 1a/2a but not with 2b

set(updated_ids__tract_area) - set(updated_ids__df_id)

{'26083000100', '41041950303', '51001090600', '51131930200', '72013301000'}

In [47]:
### VA:
# 51001090600
# https://screeningtool.geoplatform.gov/en/#9.26/37.7027/-75.8891
# 51131930200
# https://screeningtool.geoplatform.gov/en/#9.84/37.3186/-75.9043

### MI: 
# 26083000100
# https://screeningtool.geoplatform.gov/en/#8.87/47.3424/-88.1965

### OR:
# 41041950303
# https://screeningtool.geoplatform.gov/en/#11.94/45.015/-123.98189

### PR:
# 72013301000
# https://screeningtool.geoplatform.gov/en/#13.45/18.47558/-66.75585

These all look like they should be re-classified. Indicates we should not go with 2b.

In [48]:
# check the rest of the changed tracts
set(updated_ids__tract_area) & set(updated_ids__df_id)

{'12087971001',
 '12101030202',
 '12101030303',
 '12127081102',
 '26069000200',
 '28047001400',
 '39007000602',
 '48061012305',
 '55101000100',
 '72037160100'}

In [49]:
### FL:
# 12087971001
# https://screeningtool.geoplatform.gov/en/#12.3/24.73756/-81.0061
# 12101030202
# https://screeningtool.geoplatform.gov/en/#13.28/28.34631/-82.71162
# 12101030303
# https://screeningtool.geoplatform.gov/en/#12.8/28.22897/-82.75138
# 12127081102
# https://screeningtool.geoplatform.gov/en/#13.54/29.24789/-81.02269

### MI:
# 26069000200
# https://screeningtool.geoplatform.gov/en/#9.71/44.3623/-83.5055

### MS:
# 28047001400
# https://screeningtool.geoplatform.gov/en/#11.54/30.3705/-89.0511

### OH:
# 39007000602
# https://screeningtool.geoplatform.gov/en/#12.33/41.89305/-80.82678

### TX:
# 48061012305
# https://screeningtool.geoplatform.gov/en/#9.76/26.2291/-97.2455

### WI:
# 55101000100
# https://screeningtool.geoplatform.gov/en/#13.44/42.7299/-87.77935

### PR:
# 72037160100
# https://screeningtool.geoplatform.gov/en/#11.59/18.2405/-65.6222

These all look like legitimate donut holes.

#### Final recommendation: Use implementation 2a (filter the input df using ALAND10). This will allow us to update the status of all 15 of the above Census tracts, and will also let the function continue to return rows for the water tracts (as we currently do in prod).